In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Preprocessing
!ls

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1vvRy60J06fZq-dj8EPVTkgHU4gq8OCqU/Capstone-KPMG/Preprocessing
Attractions.ipynb		 I90.ipynb
Combine.ipynb			 map_request_api.py
Data				 OBSOLETE_Geo.ipynb
EV_station_count_distance.ipynb  OBSOLETE_remoteDataList
gas_station_w_traffic.csv	 __pycache__
HighwayExitDistance.ipynb	 traffic_population_crime_combination.ipynb


In [2]:
import pandas as pd
import numpy as np
from geopy import distance

In [3]:
main = pd.read_csv('Data/main_dataset.csv')
main

,gas_key,gas_name,gas_lat,gas_long,attr_cnt_1mile,attr_cnt_5mile,attr_name,attr_lat,attr_long,distance_to_nearest_attr,...,nri_risk_rating,nri_intpt_lat,nri_intpt_long,nri_zipcode,census_tract_area,census_tract_category,traff_cnt_10m_avg,traff_cnt_10m_max,traff_cnt_5m_avg,traff_cnt_5m_max
0,0,24 Hour Express Fuel,48.393760,-122.329806,0,1,Skagit County Fairgrounds,48.408145,-122.340118,1.1020,...,Relatively High,48.404815,-122.340554,98273.0,0.002191,2.0,21209.859155,72000,23733.802817,72000
1,1,76,47.191712,-120.916225,0,1,KBHR Set,47.223241,-120.993224,4.2247,...,NaN,NaN,NaN,NaN,0.000584,2.0,11678.727273,36000,9901.025641,34000
2,2,76,47.238704,-122.357577,0,4,Wagon Trail,47.275437,-122.308897,3.4174,...,Relatively High,47.249149,-122.364191,98424.0,0.001614,2.0,39766.900000,219000,43117.857143,219000
3,3,76,48.435227,-122.291324,0,1,Skagit County Fairgrounds,48.408145,-122.340118,2.9200,...,NaN,NaN,NaN,NaN,0.001533,2.0,20470.068493,72000,23101.204819,72000
4,4,76,47.839608,-120.014523,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.018331,0.0,5653.947368,12000,6723.076923,12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,2206,NaN,48.198460,-122.127847,0,1,Giant Arborvitae (Big Cedar Stump),48.168640,-122.189179,3.5008,...,Relatively Moderate,48.187437,-122.166077,98292.0,0.001817,2.0,26655.473684,123000,26604.166667,102000
2207,2207,NaN,48.556192,-122.344545,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.011283,0.0,19216.976744,72000,17606.250000,49000
2208,2208,NaN,48.747436,-122.475199,1,1,Mindport Exhibits,48.750881,-122.480674,0.3452,...,Relatively High,48.756483,-122.468299,98225.0,0.000309,2.0,21385.658915,69000,23751.250000,69000
2209,2209,NaN,48.755411,-122.463930,1,1,Mindport Exhibits,48.750881,-122.480674,0.8254,...,Relatively High,48.756483,-122.468299,98225.0,0.000309,2.0,21216.796875,69000,23464.634146,69000


In [4]:
main.columns

Index(['gas_key', 'gas_name', 'gas_lat', 'gas_long', 'attr_cnt_1mile',
       'attr_cnt_5mile', 'attr_name', 'attr_lat', 'attr_long',
       'distance_to_nearest_attr', 'crime_coord', 'crime_county',
       'crime_population', 'violent_crime', 'murder_nonnegligent_manslaughter',
       'Rape1', 'Robbery', 'aggravated_assault', 'property_crime', 'Burglary',
       'larceny_theft', 'motor_vehicle_theft', 'Arson', 'total_crime',
       'exit_name', 'exit_lat', 'exit_long', 'distance_to_nearest_exit',
       'highway', 'num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas',
       'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas',
       'num_EV_in_50_miles_of_gas', 'Closest_EV_Station_name',
       'Closest_EV_Station_lat', 'Closest_EV_Station_long',
       'distance_to_closest_ev_station', 'nri_geoid', 'nri_county',
       'nri_population', 'nri_build_value', 'nri_agri_value', 'nri_area',
       'nri_risk_score', 'nri_risk_rating', 'nri_intpt_lat', 'nri_intpt_long',
       'nri_zi

In [ ]:
def get_max_covered_traffic(locations):
  return main.iloc[locations]['traff_cnt_5m_max'].sum()

In [ ]:
def ev_station_cnt_constraints(locations):
  # number of ev stations within 2 miles < 3
  # number of ev stations within 20 miles < 10
  return (main.iloc[locations]['num_EV_in_2_miles_of_gas']<EV_CNT_LIMIT_2M).all() and (main.iloc[locations]['num_EV_in_20_miles_of_gas']<EV_CNT_LIMIT_20M).all()

In [ ]:
# TODO: check given current placement of new ev station, 50 mile requirement can be satisfied

In [ ]:
# TODO: check distance between new ev stations, should be placed more than x miles away from each other(use direct distance?)
def new_ev_position_constraints(locations):
  for i in range(len(locations)):
    for j in range(i+1, len(locations)):
      coords_1 = (main.iloc[locations[i]]['gas_lat'], main.iloc[locations[i]]['gas_long'])
      coords_2 = (main.iloc[locations[j]]['gas_lat'], main.iloc[locations[j]]['gas_long'])
      dist = distance.distance(coords_1, coords_2).miles
      if dist < EV_STATION_DIST_LIMIT: 
        # print(coords_1)
        # print(coords_2)
        # print(dist)
        return False
  return True

In [ ]:
def NRI_constraint(locations):
  # nri rating is better than relatively high
  return (main.iloc[locations]['nri_risk_rating'] != 'Relatively High').all() & (main.iloc[locations]['nri_risk_rating'] != 'Very High').all()

In [ ]:
def crime_rate_constraint(locations):
  # crime rate is lower than limit
  # TODO
  pass

In [ ]:
def check_constraints(locs):
  return NRI_constraint(locs) and new_ev_position_constraints(locs) and ev_station_cnt_constraints(locs)

# Check all combinations

In [ ]:
EV_CNT_LIMIT_2M = 3
EV_CNT_LIMIT_20M = 10
CRIME_RATE_LIMIT = 1

# get i5 only
i5_locs = main['gas_key'][main['highway'] == 'I5']
i5_stations = main.iloc[i5_locs]

# ev_station_cnt_constraints
filtered_locs = i5_stations['gas_key'][i5_stations['num_EV_in_2_miles_of_gas']<EV_CNT_LIMIT_2M]
filtered_stations = main.iloc[filtered_locs]

filtered_locs = filtered_stations['gas_key'][filtered_stations['num_EV_in_20_miles_of_gas']<EV_CNT_LIMIT_20M]
filtered_stations = main.iloc[filtered_locs]

# nri_constraint
# filtered_locs = filtered_stations['gas_key'][filtered_stations['nri_risk_rating'] != 'Relatively High']
# filtered_stations = main.iloc[filtered_locs]a

filtered_locs = filtered_stations['gas_key'][filtered_stations['nri_risk_rating'] != 'Very High']
filtered_stations = main.iloc[filtered_locs]


In [ ]:
# generate all possible combinition
import itertools
PROPOSED_STATION_CNT = 7
EV_STATION_DIST_LIMIT = 5

i5_locs = main['gas_key'][main['highway'] == 'I5']
count = 0
for subset in itertools.combinations(filtered_locs, PROPOSED_STATION_CNT):
    locs = list(subset)
    # print(locs)
    if new_ev_position_constraints(locs):
      print(locs)
      print(get_max_covered_traffic(locs))
    count += 1
    # if (count % 100 == 0): print(count)

KeyboardInterrupt: ignored

# Plot Result

In [ ]:

# for i in filtered_locs:
#   new_df = pd.DataFrame([[main.iloc[i]['gas_lat'], main.iloc[i]['gas_long'], 'Filtered', 'red', 100]], columns=result_df.columns)
#   result_df = pd.concat([result_df, new_df], ignore_index=True)

# result_df

In [26]:
import folium
def plot_result(plot_df):
  center = 46.2735210909813, -122.89553326963093

  m = folium.Map(location=center, 
                zoom_start=9,
                width=400,height=850)

  # Same as before... go through each home in set, make circle, and add to map.
  # This time we add a color using price and the colormap object
  for i in range(plot_df.shape[0]):
      folium.Circle(
          location=[plot_df.iloc[i]['Latitude'], plot_df.iloc[i]['Longitude']],
          popup=plot_df.iloc[i]['gas_key'],
          radius=800,
          fill=True,
          color = plot_df.iloc[i]['color'],
          fill_opacity=0.2
      ).add_to(m)

  return m


In [33]:
ev_station = pd.read_csv('Data/wa_EV_stations.csv')
ev_station = ev_station[['Latitude','Longitude']]
ev_station['type'] = 'EV'
ev_station['color'] = 'green'
ev_station['radius'] = 50
ev_station['gas_key'] = ''
result_df = ev_station


for i in filtered_locs:
  new_df = pd.DataFrame([[main.iloc[i]['gas_lat'], main.iloc[i]['gas_long'], 'Filtered', 'red', 100, str(i)]], columns=result_df.columns)
  result_df = pd.concat([result_df, new_df], ignore_index=True)

# result_df

# result_locs = [32, 123, 625, 1916, 1932, 2090]
result_locs = [16, 32, 123, 394, 985, 1916, 2090]
for i in result_locs:
  new_df = pd.DataFrame([[main.iloc[i]['gas_lat'], main.iloc[i]['gas_long'], 'Proposed', 'blue', 100, str(i)]], columns=result_df.columns)
  result_df = pd.concat([result_df, new_df], ignore_index=True)


# result_df
m = plot_result(result_df)
m

NameError: ignored

In [32]:
ev_station = pd.read_csv('Data/wa_EV_stations.csv')
ev_station = ev_station[['Latitude','Longitude']]
ev_station['type'] = 'EV'
ev_station['color'] = 'green'
ev_station['radius'] = 50
ev_station['gas_key'] = ''
result_df = ev_station

result_locs = [32, 123, 625, 1916, 1932, 2090]
# result_locs = [16, 32, 123, 394, 985, 1916, 2090]
for i in result_locs:
  new_df = pd.DataFrame([[main.iloc[i]['gas_lat'], main.iloc[i]['gas_long'], 'Proposed', 'blue', 100, str(i)]], columns=result_df.columns)
  result_df = pd.concat([result_df, new_df], ignore_index=True)


# result_df
m = plot_result(result_df)
m

# Testing Space

In [ ]:
test = [1,5,8,23]
print(main.iloc[test]['nri_risk_rating'])
(main.iloc[test]['nri_risk_rating'] != 'Relatively High').all() & (main.iloc[test]['nri_risk_rating'] != 'Very High').all()

In [ ]:
main['gas_key'][main['highway'] == 'I5']

In [ ]:
main.loc[1]['gas_lat']

In [ ]:
import geopy.distance
coords_1 = (47.191712,	-120.916225)
coords_2 = (47.139542,	-122.097343)
geopy.distance.distance(coords_1, coords_2).miles


In [ ]:
main['total_crime'].describe()

In [ ]:
main['gas_key']


In [15]:
def traffic_covered_gas_station(loc, ratio=1000):
  traffic_cnt = main.iloc[loc]['traff_cnt_5m_max']
  ev_cnt = main.iloc[loc]['num_EV_in_5_miles_of_gas']
  # print(loc, traffic_cnt * EV_TRAFFIC_PCT, ev_cnt)
  if ev_cnt * ratio < traffic_cnt * EV_TRAFFIC_PCT:
    return False
  return True

In [17]:
loc = 625
traffic_cnt = main.iloc[loc]['traff_cnt_5m_max']
ev_cnt = main.iloc[loc]['num_EV_in_5_miles_of_gas']


SyntaxError: ignored